In [1]:
import numpy as np
import pandas as pd
import streamlit as st
import csv
from collections import defaultdict
from numpy.random import default_rng as rng

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from os import path
from matplotlib.pyplot import figure
if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

from rsi_modules import *

mm3_url         = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=1475905714"
publication_url = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=2125519370"
theses_url      = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=609352385"

F = pd.read_csv(mm3_url)
publication = pd.read_csv(publication_url).fillna("")
theses = pd.read_csv(theses_url).fillna("")
theses = theses.drop_duplicates(subset=['รหัส (นักศึกษา)'], keep='last')

In [ ]:
st.title('ภาระงานของคณาจารย์ในสถาบันราชสุดาปีการศึกษา 2568')

st.subheader("1. ชั่วโมงสอนและจำนวนวิชาที่รับผิดชอบ")

academic_year = st.radio(
    "เลือกเทอมและปีการศึกษา",
    ["2568/1", "2568/2"],
)

if academic_year == "2568/1":
    start_date = '8/5/2025'
    end_date   = '9/19/2025'
elif academic_year == "2568/2":
    start_date = '12/12/2025'
    end_date   = '12/30/2025'

f = F[F['Timestamp'].between(start_date, end_date)]
t1,t2 = report_mm3(f)

st.subheader("2. ผลงานตีพิมพ์ทางวิชาการในแต่ละปี")
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
st.write(df)

st.subheader("3. ภาระงานควบคุมวิทยานิพนธ์")
B = pd.DataFrame(get_thesis_workload_new(theses)).T
st.write(B)

st.subheader("4. ภาพรวมความก้าวหน้าวิทยานิพนธ์ของนักศึกษา")
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)
st.write(A)
st.write("หมายเหตุ:")
st.write("MA  = หส.ปริญญาโท สาขาคุณภาพชชีวิตคนพิการ")
st.write("MEd = หส.ปริญญาโท สาขาการศึกษาพิเศษ")
st.write("PhD = หส.ปรัชญาดุษฎีบัณฑิต สาขาคุณภาพชีวิตคนพิการ")

In [2]:
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)

In [3]:
st.write(A)

2025-12-17 17:05:31.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 17:05:31.468 
  command:

    streamlit run c:\Users\User\miniconda3\envs\rsi_dashboard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-17 17:05:31.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 17:05:31.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
